In [5]:
import json
import os

In [6]:
fname="extended_train_data"

In [2]:
train_file = "train.json"
dev_file = "dev.json"
test_file = "test-unlabelled.json"

In [41]:
def get_text_from_json(file_path):
    texts = []
    with open(file_path) as json_file:
        data = json.load(json_file)
        for key in data.keys():
            texts.append(data[key]["text"])
    return texts

In [42]:
train_texts = get_text_from_json(train_file_path)
dev_texts = get_text_from_json(dev_file_path)
test_texts = get_text_from_json(test_file_path)

In [43]:
print(len(train_texts))
print(len(dev_texts))
print(len(test_texts))

1168
100
1410


In [44]:
def get_extended_train_text(fname):
    extend_train_texts= []
    for file in sorted(os.listdir(fname)):
        if ".DS_Store" not in file:
            with open(os.path.join(fname, file)) as json_file:
                data = json.load(json_file)
                texts = data["pages"]
                for text in texts:
                    if "text" in text.keys():
                        extend_train_texts.append(text["text"])
    return extend_train_texts

In [45]:
extend_train_texts = get_extended_train_text(fname)
extend_train_labels = [0]*len(extend_train_texts)

In [46]:
def get_label_from_json(file_path):
    labels = []
    with open(file_path) as json_file:
        data = json.load(json_file)
        for key in data.keys():
            labels.append(data[key]["label"])
    return labels

In [47]:
train_labels = get_label_from_json(train_file_path)
dev_labels = get_label_from_json(dev_file_path)

In [48]:
print(len(train_labels))
print(len(dev_labels))

1168
100


In [49]:
train_texts += extend_train_texts
train_labels += extend_train_labels

In [50]:
print(len(train_texts))
print(len(train_labels))

2182
2182


In [17]:
with open("train_file.txt","w") as file:
    for index in range(len(train_texts)):
        file.write(str("__label__")+str(train_labels[index])+" "+train_texts[index].replace("\n", " ")+"\n")

In [18]:
with open("dev_file.txt","w") as file:
    for index in range(len(dev_texts)):
        file.write(str("__label__")+str(dev_labels[index])+" "+dev_texts[index].replace("\n", " ")+"\n")

In [19]:
with open("dev_unlabelled_file.txt","w") as file:
    for index in range(len(dev_texts)):
        file.write(dev_texts[index].replace("\n", " ")+"\n")

In [20]:
with open("test_file.txt","w") as file:
    for index in range(len(test_texts)):
        file.write(test_texts[index].replace("\n", " ")+"\n")

In [30]:
def get_label_from_text(file_path):
    labels = []
    with open(file_path) as file:
        label = file.readline()
        while label: 
            if "__label__" in label:
                labels.append(int(label.strip("__label__")))
            else:
                labels.append(int(label))
            label = file.readline()
    return labels

In [22]:
from sklearn.metrics import precision_recall_fscore_support
def compute_F1(predict_label, true_label):
    p, r, f, _ = precision_recall_fscore_support(true_label, predict_label, pos_label=1, average="binary")
    return f

In [38]:
predict_dev_labels = get_label_from_text("fastText/prediction/dev_lr0.5_epoch100.txt")
print(compute_F1(predict_dev_labels, dev_labels))

0.7377049180327869


In [39]:
predict_dev_labels = get_label_from_text("fastText/prediction/dev_lr0.5_epoch1000.txt")
print(compute_F1(predict_dev_labels, dev_labels))

0.7479674796747968


In [ ]:
predict_dev_labels = get_label_from_text("fastText/prediction/dev_lr0.5_epoch2000.txt")
print(compute_F1(predict_dev_labels, dev_labels))

In [34]:
predict_dev_labels = get_label_from_text("fastText/prediction/dev_lr0.5_epoch100_word1Gram.txt")
print(compute_F1(predict_dev_labels, dev_labels))

0.7419354838709677


In [36]:
predict_dev_labels = get_label_from_text("fastText/prediction/dev_lr0.5_epoch1000_word1Gram.txt")
print(compute_F1(predict_dev_labels, dev_labels))

0.7580645161290321


In [37]:
predict_dev_labels = get_label_from_text("fastText/prediction/dev_lr0.5_epoch2000_word1Gram.txt")
print(compute_F1(predict_dev_labels, dev_labels))

0.7580645161290321


In [27]:
print(compute_F1([1]*100, dev_labels))

0.6666666666666666


In [ ]:
# Mac command to install and run fastText
git clone https://github.com/facebookresearch/fastText.git
cd fastText
make
./fasttext supervised -input train_file.txt -output model1_lr0.5_epoch100_word1Gram -lr 0.5 -epoch 100 -wordNgrams 1
./fasttext test model1_lr0.5_epoch100_word1Gram.bin dev_file.txt
./fasttext predict model1_lr0.5_epoch100_word1Gram.bin dev_unlabelled_file.txt > ../prediction/dev_lr0.5_epoch100_word1Gram.txt